In [98]:
# !pip install pandas google-cloud-storage
# !pip freeze > requirements.txt

In [99]:
import pandas as pd

In [100]:
import os
from google.cloud import storage

# Set GCS credentials (Use absolute path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/creds.json"

# GCS Configuration
BUCKET_NAME = "indian_road_accidents-sandbox-449108"
storage_client = storage.Client()
storage_client

In [101]:
# Read CSV without specifying schema initially
df_sample = pd.read_csv("../data/state-wise-accidents.csv", nrows=5)  # Read first few rows

# Identify all year columns dynamically (Assuming they are numeric)
years = [col for col in df_sample.columns if col.isnumeric()]

# Build dynamic schema
state_schema = {"States/UTs": "string"}  # Keep categorical column as string
state_schema.update({year: "Int64" for year in years})  # Assign Int64 to all year columns

print(state_schema)  # Debugging - See the generated schema

{'States/UTs': 'string', '2018': 'Int64', '2019': 'Int64', '2020': 'Int64', '2021': 'Int64', '2022': 'Int64'}


In [102]:
state = pd.read_csv("../data/state-wise-accidents.csv", index_col=0, dtype=state_schema)
state.head()

,States/UTs,2018,2019,2020,2021,2022
Sl. No.,,,,,,
1,Andhra Pradesh,24475,21992,19509,21556,21249
2,Arunachal Pradesh,277,237,134,283,227
3,Assam,8248,8350,6595,7411,7023
4,Bihar,9600,10007,8639,9553,10801
5,Chhattisgarh,13864,13899,11656,12375,13279


In [103]:
state.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 1 to 37
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   States/UTs  37 non-null     string
 1   2018        36 non-null     Int64 
 2   2019        36 non-null     Int64 
 3   2020        35 non-null     Int64 
 4   2021        36 non-null     Int64 
 5   2022        36 non-null     Int64 
dtypes: Int64(5), string(1)
memory usage: 2.2 KB


In [104]:
years = [col for col in state.columns if col.isnumeric()]
years

['2018', '2019', '2020', '2021', '2022']

In [105]:
state.head()

,States/UTs,2018,2019,2020,2021,2022
Sl. No.,,,,,,
1,Andhra Pradesh,24475,21992,19509,21556,21249
2,Arunachal Pradesh,277,237,134,283,227
3,Assam,8248,8350,6595,7411,7023
4,Bihar,9600,10007,8639,9553,10801
5,Chhattisgarh,13864,13899,11656,12375,13279


In [106]:
for year in years:
    df_year = state[["States/UTs", year]].rename(columns={year: "Accidents"})
    df_year["year"] = year  # Add the year column
    
    # Reorder columns to have "year" first
    df_year = df_year[["year", "States/UTs", "Accidents"]]

    # Convert DataFrame to CSV string
    csv_data = df_year.to_csv(index=False)

    # Upload to GCS
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(f"state_wise_road_accidents/raw/{year}/{year}.csv")
    blob.upload_from_string(csv_data, content_type="text/csv")

    print(f"Uploaded {year}.csv to GCS under state_wise_road_accidents/raw/{year}/")

print("All files uploaded successfully!")

Uploaded 2018.csv to GCS under state_wise_road_accidents/raw/2018/
Uploaded 2019.csv to GCS under state_wise_road_accidents/raw/2019/
Uploaded 2020.csv to GCS under state_wise_road_accidents/raw/2020/
Uploaded 2021.csv to GCS under state_wise_road_accidents/raw/2021/
Uploaded 2022.csv to GCS under state_wise_road_accidents/raw/2022/
All files uploaded successfully!


In [107]:
# Read CSV without specifying schema initially
df_sample = pd.read_csv("../data/deaths-by-accident-type.csv", nrows=5)  # Read first few rows

# Identify all year columns dynamically (Assuming they are numeric)
years = [col for col in df_sample.columns if col.isnumeric()]

# Build dynamic schema
deaths_schema = {"Type of Collision": "string"}  # Keep categorical column as string
deaths_schema.update({year: "Int64" for year in years})  # Assign Int64 to all year columns

print(deaths_schema)  # Debugging - See the generated schema

{'Type of Collision': 'string', '2018': 'Int64', '2019': 'Int64', '2020': 'Int64', '2021': 'Int64', '2022': 'Int64'}


In [108]:
deaths = pd.read_csv("../data/deaths-by-accident-type.csv", index_col=0, dtype=deaths_schema)
deaths

,Type of Collision,2018,2019,2020,2021,2022
Sl. No.,,,,,,
1,Hit and Run,28619,29354,23159,25938,30486
2,With parked Vehicle,4780,5086,4243,4925,6012
3,Hit from Back,25801,27765,22989,28712,32907
4,Hit from side,15477,16342,14271,18299,20357
5,Run off Road,7731,8419,7782,9150,9862
6,Fixed object,4623,5739,5039,6600,7307
7,Vehicle overturn,9548,9011,8611,9122,9827
8,Head on collision,29646,26772,25289,27248,26413
9,Others,25192,22625,20331,23978,25320


In [109]:
deaths.dtypes

Type of Collision    string[python]
2018                          Int64
2019                          Int64
2020                          Int64
2021                          Int64
2022                          Int64
dtype: object

In [110]:
years = [col for col in deaths.columns if col.isnumeric()]
years

['2018', '2019', '2020', '2021', '2022']

In [111]:
year = years[1]
df_year = deaths[["Type of Collision", year]].rename(columns={year: "Deaths"})
df_year["year"] = year
df_year

,Type of Collision,Deaths,year
Sl. No.,,,
1,Hit and Run,29354,2019
2,With parked Vehicle,5086,2019
3,Hit from Back,27765,2019
4,Hit from side,16342,2019
5,Run off Road,8419,2019
6,Fixed object,5739,2019
7,Vehicle overturn,9011,2019
8,Head on collision,26772,2019
9,Others,22625,2019


In [112]:
for year in years:
    df_year = deaths[["Type of Collision", year]].rename(columns={year: "Deaths"})
    df_year["year"] = year  # Add the year column
    
    # Reorder columns to have "year" first
    df_year = df_year[["year", "Type of Collision", "Deaths"]]

    # Convert DataFrame to CSV string
    csv_data = df_year.to_csv(index=False)

    # Upload to GCS
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(f"deaths_by_road_accidents/raw/{year}/{year}.csv")
    blob.upload_from_string(csv_data, content_type="text/csv")

    print(f"Uploaded {year}.csv to GCS under deaths_by_road_accidents/raw/{year}/")

print("All files uploaded successfully!")

Uploaded 2018.csv to GCS under deaths_by_road_accidents/raw/2018/
Uploaded 2019.csv to GCS under deaths_by_road_accidents/raw/2019/
Uploaded 2020.csv to GCS under deaths_by_road_accidents/raw/2020/
Uploaded 2021.csv to GCS under deaths_by_road_accidents/raw/2021/
Uploaded 2022.csv to GCS under deaths_by_road_accidents/raw/2022/
All files uploaded successfully!
